In [1]:
%load_ext autoreload
%autoreload 2
import mycode.vap as vap

In [2]:
# The goal is to run new vespa and check a rank profile always return match features, even when `select some_field from sources * ...` is specified.
# Then run some year-old version, and check if behavior is the same.

In [3]:
# Create a sample VAP
from vespa.package import (ApplicationPackage, Field, Schema, Document, RankProfile, Function)

doc_schema = Schema(
    name="doc",
    document=Document(
        fields=[
            Field(
                name="id",
                type="int",
                indexing=["attribute"],
                attribute=["fast-search"],
            ),
        ]
    ),
    rank_profiles=[
        RankProfile(
            name='fields',
            inherits='unranked',
            first_phase="0",
            functions=[
                Function(
                    name='id',
                    expression='attribute(id)'
                )
            ],
            match_features=[
                'id'
            ],
        )
    ]
)

application_package = ApplicationPackage(
    name="test",
    schema=[doc_schema],
)

In [6]:
from vespa.deployment import VespaDocker

# In case running colima on macos run the following
# !sudo ln -sf $HOME/.colima/default/docker.sock /var/run/docker.sock
vespa_docker = VespaDocker(
    container_image="vespaengine/vespa:8.637.17",
)

In [8]:
# Start a docker container and deploy the application package
client = vespa_docker.deploy(
    application_package=application_package,
)

Application is up!
Finished deployment.


In [9]:
# Create and feed 1 dummy doc
docs = [
    {
        'id': f'{1}',
        'fields': {
            'id': 1,
        }
    }
]

client.feed_iterable(docs, schema="doc", namespace="doc", callback=vap.feed_callback)

In [11]:
client.query(body={
    'yql': 'select * from sources * where true',
    'ranking.profile': 'fields',
}).json

{'root': {'children': [{'fields': {'documentid': 'id:doc:doc::1',
     'matchfeatures': {'id': 1.0},
     'sddocname': 'doc'},
    'id': 'id:doc:doc::1',
    'relevance': 0.0,
    'source': 'test_content'}],
  'coverage': {'coverage': 100,
   'documents': 1,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'fields': {'totalCount': 1},
  'id': 'toplevel',
  'relevance': 1.0}}

In [12]:
# matchfeatures are returned as expected

In [13]:
client.query(body={
    'yql': 'select id from sources * where true',
    'ranking.profile': 'fields',
}).json

{'root': {'children': [{'fields': {'id': 1, 'matchfeatures': {'id': 1.0}},
    'id': 'index:test_content/0/c4ca42388ce70a10b392b401',
    'relevance': 0.0,
    'source': 'test_content'}],
  'coverage': {'coverage': 100,
   'documents': 1,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'fields': {'totalCount': 1},
  'id': 'toplevel',
  'relevance': 1.0}}

In [14]:
# and match features are also returned even if only the `id` is explicitly asked

In [20]:
!docker stop {vespa_docker.container_id}

3e32df902179a04a32ad18997d06df8adb9f3a21b9d507bc9752cda1fa471fd8


In [34]:
vespa_docker_old = VespaDocker(
    container_image="vespaengine/vespa:8.399.59",
)

In [35]:
client_old = vespa_docker_old.deploy(
    application_package=application_package,
)

Application is up!
Finished deployment.


In [29]:
# Create and feed 1 dummy doc
docs = [
    {
        'id': f'{1}',
        'fields': {
            'id': 1,
        }
    }
]

client_old.feed_iterable(docs, schema="doc", namespace="doc", callback=vap.feed_callback)

In [36]:
client_old.query(body={
    'yql': 'select * from sources * where true',
    'ranking.profile': 'fields',
}).json

{'root': {'children': [{'fields': {'documentid': 'id:doc:doc::1',
     'matchfeatures': {'id': 1.0},
     'sddocname': 'doc'},
    'id': 'id:doc:doc::1',
    'relevance': 0.0,
    'source': 'test_content'}],
  'coverage': {'coverage': 100,
   'documents': 1,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'fields': {'totalCount': 1},
  'id': 'toplevel',
  'relevance': 1.0}}

In [37]:
client_old.query(body={
    'yql': 'select id from sources * where true',
    'ranking.profile': 'fields',
}).json

{'root': {'children': [{'fields': {'id': 1, 'matchfeatures': {'id': 1.0}},
    'id': 'index:test_content/0/c4ca42388ce70a10b392b401',
    'relevance': 0.0,
    'source': 'test_content'}],
  'coverage': {'coverage': 100,
   'documents': 1,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'fields': {'totalCount': 1},
  'id': 'toplevel',
  'relevance': 1.0}}

In [38]:
!docker stop {vespa_docker_old.container_id}

3e32df902179a04a32ad18997d06df8adb9f3a21b9d507bc9752cda1fa471fd8


In [ ]:
# It turns out that both old and new return match-features, even if only some fields are specified.